In [44]:
from iron import phi
import numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization, UtilityFunction
from sklearn.svm import SVC
import pandas as pd
import warnings
import random
import math

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression



warnings.filterwarnings("ignore")

# take 80% train, use random forest for model, then select a case from test set, then run
predictor = "T_degC"

# returns features, X: features and y: the feature we predict (predictor)
def prepare_data():
    ######### IMPORT DATA # ########
    file = '/Users/johnkim/Downloads/archive (1)/bottle.csv'
    df = pd.read_csv(file)
    pd.set_option('display.max_columns', None) # view all cols

    # REMOVE COLUMNS WITH MORE THAN 10% NA VALUES
    min_na = len(df) * .1 # removes all cols with more than 10% NA values
    df = df.dropna(thresh=min_na, axis=1)
    df = df.drop(['Sta_ID', 'Depth_ID', 'Cst_Cnt', 'Btl_Cnt', 'BtlNum', 'RecInd','T_prec'], axis=1)
    # df.head(10)
    # print(df.head(10))

    # LIMITS FEATURES TO ONLY 10
    features = list(df.columns.values) # get features // limited columns
    # print(features)
    features = features[0:6]
    new_df = df[features].copy().dropna() # same num rows
    print(features)

    # remove feature we want to predict
    features.pop(2)
    print(features)
    # features = features[0, 1:6]
    df.reset_index(drop=True, inplace=True)


    # FEATURES
    X = new_df.loc[:, features] 
    # PREDICTOR
    y = new_df.loc[:, [predictor]]
    
    return features, X, y

def ttsplit(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=.80)
    # reg = RandomForestRegressor(max_depth = 2, random_state = 0)
    # reg.fit(X_train, y_train)
    return X_train, X_test, y_train, y_test

# returns phi relevance function and the relevance numpy array of the predictor (y)
# CHECK WITH NUNO
def get_phis(y_train, y_test):
    ph = phi.phi_control(y_train[predictor], extr_type="both")
    phis = phi.phi(y_test[predictor], phi_parms=ph)
    # print(phis)
    print(y_test[predictor])
    print(type(y_test[predictor]))
    # phis[1]
    return ph, phis

base = [] # original feature values
pbounds = {}

# returns regression model, y original, and y relevance
def create_regressor(features, X_train, X_test, y_train, y_test, phis):
    reg = RandomForestRegressor(n_estimators=500, random_state = 0) # n = 500
    # reg = LinearRegression().fit(X_train, y_train)
    # svm regression or multilayer small neural network (multilayer perceptron)
    # random forest
    # try mars 
    # try lightgbm 
    reg.fit(X_train, y_train)
    
    # NOT NECESSARY
    
    
    i = random.randint(0, len(X_test)) # random variable
    
    # actual y value
    y_orig = y_test.iloc[i][predictor]
    # y relevance value
    y_rel = phis[i]
    
    # creating pbounds based on mean of features
    for f in features: 
        x = X_test.iloc[i][f]
        base.append(x)
        sd = 0.05 # calculates sd 
        pbounds[f] = (x - (sd * x), x + (sd * x))
        

    return base, pbounds, reg, y_orig, y_rel



def create_vars():
    i = random.randint(0, len(X_test)) # random variable

    # actual y value
    y_orig = y_test.iloc[i][predictor]
    # y relevance value
    y_rel = phis[i]
    
    # creating pbounds based on mean of features
    for f in features: 
        x = X_test.iloc[i][f]
        base.append(x)
        sd = 0.05
        pbounds[f] = (x - (sd * x), x + (sd * x))
    
    return base, pbounds, y_orig, y_rel


features, X, y = prepare_data()
X_train, X_test, y_train, y_test = ttsplit(X, y)
# reg, y_orig, y_rel = create_regressor(features, X_train)
ph, phis = get_phis(y_train, y_test)
base, pbounds, reg, y_orig, y_rel = create_regressor(features, X_train, X_test, y_train, y_test, phis)

# x_orig = 15
# y_orig = 34





['Depthm', 'T_degC', 'Salnty', 'O2ml_L', 'STheta', 'O2Sat']
['Depthm', 'T_degC', 'O2ml_L', 'STheta', 'O2Sat']
691628    14.99
828063     7.22
753076     9.39
530014     9.54
300976     5.64
          ...  
390087    13.80
813764     6.49
522940    11.57
757369    15.67
827470    12.46
Name: T_degC, Length: 132254, dtype: float64
<class 'pandas.core.series.Series'>


In [ ]:

def run_all_abs_y():
    pdiffs_abs_y = []
    params = []

    for i in range(0, len(X_test), 10):
        print(i)    
        # actual y value
        y_orig = y_test.iloc[i][predictor]
        print(y_orig)
        # y relevance value
        # y_rel = phis[i]
        
        # creating pbounds based on mean of features
        for f in features: 
            x = X_test.iloc[i][f]
            base.append(x)
            sd = 0.05
            pbounds[f] = (x - (sd * x), x + (sd * x))
            
            
        def black_box_function_abs_y(Depthm, T_degC, Salnty, O2ml_L, STheta, O2Sat, BtlNum, RecInd, T_prec):
    
            # when generating x, we will have a list of vars with all the numeric and categorical
            
            x = [[BtlNum, Depthm, O2Sat, O2ml_L, RecInd, STheta, Salnty, T_degC, T_prec]]
            print("x:", x)

            y1 = reg.predict(x)    
            print("y1:", y1)
            pdiff = abs(y1[0] - y_orig)
                
            print(pdiff)
            return pdiff[0]
        

        ########### CREATING BAYESIAN OPTIMIZER ###########
        optimizer_abs_y = BayesianOptimization(
            f=black_box_function_abs_y,  # f(x)
            pbounds=pbounds,  # parameter bounds
            random_state=1,
            allow_duplicate_points=True,
        )

        ########### MAXIMIZING (finding optimums) ###########
        optimizer_abs_y.maximize(
            init_points=5,  # num of random steps to perform --> can help diversify explore space
            n_iter=50,  # number of iterations/steps of baye opt --> more = more accurate optimum
        )
                
        print(optimizer_abs_y.max['target'], optimizer_abs_y.max['params'])
        print("target")
        print(optimizer_abs_y.max['target'])
        print("params")
        print(optimizer_abs_y.max['params'])
        pdiffs_abs_y.append(optimizer_abs_y.max['target'])
        params.append(optimizer_abs_y.max['params'])
        
    return pdiffs_abs_y, params
    
pdiffs_abs_y, params = run_all_abs_y()
# [15.2, 80.75, 55.385, 3.7904999999999998, 2.85, 24.396, 31.985549999999996, 11.907, 1.9]
# [15.2, 80.75, 55.38, 3.79, 2.85, 24.4, 31.99, 11.91, 1.9]


In [ ]:

# see prediction of the y value of that new case
    # then find the y rel for that new case
# repeat in the prediction model with original and new y rel

def run_all_abs_rel():
    pdiffs_abs_rel = []
    params_rel = []

    for i in range(0, len(X_test), 10):
        print(i)
        
        # actual y value
        # y_orig = y_test.iloc[i][predictor]
        # y relevance value
        y_rel = phis[i]
        print(y_test.iloc[i][predictor])
        
        # creating pbounds based on mean of features
        for f in features: 
            x = X_test.iloc[i][f]
            base.append(x)
            sd = 0.05
            pbounds[f] = (x - (sd * x), x + (sd * x))
            
            
        def black_box_function_abs_rel(Depthm, T_degC, Salnty, O2ml_L, STheta, O2Sat, BtlNum, RecInd, T_prec):
            # when generating x, we will have a list of vars with all the numeric and categorical
            x = [[BtlNum, Depthm, O2Sat, O2ml_L, RecInd, STheta, Salnty, T_degC, T_prec]]
            print(x)
            
            y1 = reg.predict(x)
            print(y1[0][0])
            
            y_rel_pred = phi.phi(pd.Series(y1[0][0]), phi_parms=ph)
            print("rel: ", y_rel_pred)
            # print(phi.phi(pd.Series(y_test.iloc[i][predictor]), phi_parms=ph))
            
            # relevance - relevance of predicted y
            pdiff = abs(y_rel - y_rel_pred[0])
            # print(pdiff)
            return pdiff
        
        
        ########### CREATING BAYESIAN OPTIMIZER ###########
        optimizer_abs_rel = BayesianOptimization(
            f=black_box_function_abs_rel,  # f(x)
            pbounds=pbounds,  # parameter bounds
            random_state=1,
            allow_duplicate_points=True,
        )

        ########### MAXIMIZING (finding optimums) ###########
        optimizer_abs_rel.maximize(
            init_points=5,  # num of random steps to perform --> can help diversify explore space
            n_iter=25,  # number of iterations/steps of baye opt --> more = more accurate optimum
        )
        
        pdiffs_abs_rel.append(optimizer_abs_rel.max['target'])
        params_rel.append(optimizer_abs_rel.max['params'])
        
        # print(optimizer_abs_rel.max['target'])
        # print(optimizer_abs_rel.max['params'])
        
        print(optimizer_abs_rel.max['target'], optimizer_abs_rel.max['params'])
        print("target")
        print(optimizer_abs_rel.max['target'])
        print("params")
        print(optimizer_abs_rel.max['params'])

        
    return pdiffs_abs_rel, params_rel
    
pdiffs_abs_rel, params_rel = run_all_abs_rel()



In [ ]:
# ONLY HAVE TO DO IF WE RUN THE MODEL AGAIN WITH UPDATES

def convert_to_dfs():
    df1 = pd.DataFrame(pdiffs_abs_y)
    # df6.rename(columns={0: "Difference"})
    # df1
    df1.to_csv("difference_no_relevance.csv")    
    # df1.to_csv("new.csv")


    df2 = pd.DataFrame(pdiffs_abs_rel)
    # df2.rename(columns={0: "Difference"})
    # df2
    df2.to_csv("difference_yes_relevance.csv")

    df3= pd.DataFrame(params)
    # df3
    df3.to_csv("new_f.csv")

    df3.to_csv("features_no_relevance.csv")

    df4 = pd.DataFrame(params_rel)
    # df4
    df4.to_csv("features_yes_relevance.csv")

convert_to_dfs()

In [ ]:
import csv

f_abs_file = "/Users/johnkim/Developer/features_no_relevance.csv"
# f_abs_file = "/Users/johnkim/Developer/new_f.csv"

# f_rel_file = "/Users/johnkim/Developer/features_yes_relevance.csv"
pdiffs_abs_file = "/Users/johnkim/Developer/difference_no_relevance.csv"
pdiffs_abs_file = "/Users/johnkim/Developer/new.csv"

pdiffs_rel_file = "/Users/johnkim/Developer/difference_yes_relevance.csv"

def read_file(file):
    arr = []
    with open(file, newline='') as f:
        reader = csv.reader(f)
        for lines in reader:
            if (len(lines) == 2): arr.append(float(lines[1]))
            else: 
                try:
                    arr.append(list(np.float_(lines[1:])))
                except ValueError:
                    pass
                    
        
        if arr[0] == 0: arr.pop(0)
        
        # for a in arr:
        #     list(np.float_(a))
            # for num in a:
                # list(np.float_(list_name)
                # num = float(num)

    return arr

pdiffs_abs = read_file(pdiffs_abs_file)
pdiffs_rel = read_file(pdiffs_rel_file)
features_abs = read_file(f_abs_file)
features_rel = read_file(f_rel_file)

# features_rel.pop(0) # since first entry is just 0, 1, 2, 3, 4,5 

print(pdiffs_abs)
print(pdiffs_rel)
print(features_abs)
# print(len(feature_abs))
print(features_rel)

In [ ]:
# through the optimized features --> predict the corresponding y val --> convert to phi (relevance)
# this abs(relevance - actual relevance) is returned
def calculate_relevance_difference(params, pdiffs): 
    difference_of_relevance = []

    for i in range(len(pdiffs)):
        x_param = [params[i]]
                
        # predict y from the optimized test case --> calculate phi from it
        y_pred = reg.predict(x_param)[0]
        y_pred_rel = phi.phi(pd.Series(y_pred[0]), phi_parms=ph)
        
        # true y --> calculate phi
        y_true_rel = phis[i * 10]
        # y_true = y_test.iloc[i * 10][predictor]
        # y_true_rel = phi.phi(pd.Series(y_true), phi_parms=ph)
        
        relevance_difference = abs(y_pred_rel - y_true_rel)
        difference_of_relevance.append(relevance_difference[0])
    
    return difference_of_relevance


# THIS IS FOR THE BO RELEVANCE BECAUSE WE OPTIMIZED THE MODEL TO FIND RELEVANCE DIFFERENCE
# THE PDIFF RETURNED IS REL DIFF --> MUST CALCULATE OPTIMIZED Y INSTEAD OF REL

# through the optimized features --> predict corresponding y val -->
# find absolute difference between this predicted value and the actual y val 
def calculate_y_difference(params, pdiffs):
    difference_of_y = []
    
    for i in range(len(pdiffs)):
        x_param = [params[i]]

        # predict y from the optimized test case BEST CASE FROM PREDICTION
        y_pred = reg.predict(x_param)[0]
        y_orig = y_test.iloc[i * 10][predictor]
        
        abs_diff = abs(y_pred - y_orig)
        difference_of_y.append(abs_diff[0])
    
    return difference_of_y


difference_of_relevance_abs_y = calculate_relevance_difference(features_abs, pdiffs_abs)
difference_of_relevance_abs_rel_y = calculate_relevance_difference(features_rel, pdiffs_rel)

print(difference_of_relevance_abs_y)
print(difference_of_relevance_abs_rel_y)

the_difference_of_y = calculate_y_difference(features_rel, pdiffs_rel)

print(the_difference_of_y)

In [ ]:
# # plot 1
# pdiffs_abs_y
# pdiffs_abs_rel


# plot_data = [pdiffs_abs_y, pdiffs_abs_rel]
plot_data = [pdiffs_abs, the_difference_of_y]

 
fig = plt.figure(figsize =(10, 7))
 
# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xticklabels(['BO Y Difference', 'BO Y Relevance Difference'])
ax.set_ylabel('Y true - Y predicted')
 
# Creating plot
bp = ax.boxplot(plot_data)
 
# show plot
plt.show()

# plot 2
# phi_diff_rel = the relevance difference between the actual y AND the precited y (from optimized test case) OF MODEL 1
plot_data1 = [difference_of_relevance_abs_y, difference_of_relevance_abs_rel_y]
 
fig = plt.figure(figsize =(10, 7))
 
# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xticklabels(['BO Y Difference', 'BO Y Relevance Difference'])
ax.set_ylabel('Y true - Y predicted (Relevance)')
 
# Creating plot
bp = ax.boxplot(plot_data1)
 
# show plot
plt.show()


# plot_data2 = [difference_of_relevance_abs_y, pdiffs_rel]
 
# fig = plt.figure(figsize =(10, 7))
 
# # Creating axes instance
# ax = fig.add_axes([0, 0, 1, 1])
# ax.set_xticklabels(['BO Y Difference', 'BO Y Relevance Difference'])
# ax.set_ylabel('Y true - Y predicted (Relevance)')
 
# # Creating plot
# bp = ax.boxplot(plot_data2)
 
# # show plot
# plt.show()

# bayes opt reducing min
# bayes opt reducing rel between predicted and true

# BO with relevance is better because median is lower --> closer distance, lower cost
# ends probbly steep case where small change create large difference
# slightly greate rrelvance difference for rel BO

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
# find distance between actual and predicted params (features)

params_orig = []
X_f = X_test.values.tolist()
print(X_f)
for i in range(0, len(X_test), 10):
    params_orig.append(X_f[i])

# print(X_test)
print(params_orig)
print(len(params_orig))



f1 = np.array(features_abs)
f2 = np.array(features_rel)

distance_abs = euclidean_distances(params_orig, f1)
distance_rel = euclidean_distances(params_orig, f2)

d_abs = []
d_rel = []

for d in distance_abs:
    if np.mean(d) > 1000: continue
    d_abs.append(np.mean(d))
    
for d in distance_rel:
    if np.mean(d) > 1000: continue
    d_rel.append(np.mean(d))
    
print(d_abs)
print(d_rel)

relative_diff = []
for i in range(len(d_rel)):
    relative_diff.append(((d_rel[i] - d_abs[i]) / d_abs[i]) * 100)

# relative_diff = (d_rel - d_abs) / d_abs
print(relative_diff)


In [ ]:
data = [relative_diff]
 
fig = plt.figure(figsize =(10, 7))
 
# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xticklabels(['Relative Percentual Difference'])
ax.set_ylabel('Algorithmic Recourse (Y - Yt)')
 
# Creating plot
bp = ax.boxplot(data)
 
# show plot
plt.show()
